In [1]:
import tensorflow as tf
import tensorflow3d as t3d
import numpy as np

/usr/local/lib/python3.6/dist-packages/tensorflow3d/python/ops/_fps_ops.so


In [2]:
pt_sample=np.random.rand(1, 2048, 3).astype('float32')
flownet3d = t3d.models.FlowNet3D(name='flownet3d').build(input_shape1=(2048, 3), input_shape2=(2048, 3))
flownet3d.summary()

pred = flownet3d((pt_sample, pt_sample))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2048, 3)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2048, 3)]    0                                            
__________________________________________________________________________________________________
set_conv (SetConv)              ((None, 1024, 3), (N 3808        input_1[0][0]                    
__________________________________________________________________________________________________
set_conv_2 (SetConv)            ((None, 1024, 3), (N 3808        input_2[0][0]                    
_______________________________________________________________________________________

In [3]:
pred

(<tf.Tensor: shape=(1, 256, 3), dtype=float32, numpy=
 array([[[1.01285307e-02, 3.94430429e-01, 2.35697642e-01],
         [9.66822147e-01, 9.03236330e-01, 9.48838353e-01],
         [9.80848968e-01, 5.74826859e-02, 1.85185634e-02],
         [8.86522055e-01, 9.98978972e-01, 1.83724565e-03],
         [5.86025834e-01, 5.30319996e-02, 9.04713392e-01],
         [1.50173465e-02, 9.30765092e-01, 9.88044798e-01],
         [7.27662802e-01, 5.24050415e-01, 4.46304649e-01],
         [2.06686065e-01, 9.60386992e-01, 8.78770649e-03],
         [2.59597879e-03, 2.03302100e-01, 8.84830356e-01],
         [4.58866030e-01, 4.01730575e-02, 3.54812264e-01],
         [4.05552179e-01, 9.78306830e-01, 5.89074135e-01],
         [4.95568722e-01, 6.22564137e-01, 9.80114520e-01],
         [9.92820740e-01, 2.59781312e-02, 5.77095628e-01],
         [9.49003220e-01, 4.11286861e-01, 9.93067205e-01],
         [4.18350250e-01, 4.89503711e-01, 2.74149608e-03],
         [2.22486723e-03, 7.35173166e-01, 5.58028281e-01],
  